<a href="https://colab.research.google.com/github/felipeeng23/viewsbancodedados/blob/main/Felipe_SQL_Avolta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mysql-connector-python

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 15.9 MB/s eta 0:00:00


In [2]:
import mysql.connector
import pandas as pd

In [3]:
host= '34.46.77.89' #IP público da instância
user= 'root'
password= ''
database= 'Locadora'

In [4]:
!curl ipecho.net/plain

34.16.165.203

In [5]:
def consulta(query, tabela):
  connection = mysql.connector.connect(
      host = host,
      user = user,
      password = password,
      database = database
  )
  cursor = connection.cursor()

  try:
    cursor.execute(query)
    result = cursor.fetchall()
    globals()[tabela] = pd.DataFrame(result, columns=cursor.column_names)
    display(globals()[tabela])
  finally:
    cursor.close()
    connection.close()

In [6]:
def ddl(comando):
    connection = mysql.connector.connect(
        host=host,
        user=user,
        password=password,
        database=database
    )
    cursor = connection.cursor()

    try:
        cursor.execute(comando)
        connection.commit()  # Confirma as mudanças no banco
        print(f"Comando DDL executado com sucesso:\n{comando}")
    except mysql.connector.Error as err:
        print(f"Erro ao executar o comando DDL: {err}")
    finally:
        cursor.close()
        connection.close()

#GROUP_CONCAT

In [7]:
''' Liste os estados onde os clientes alugaram carros e os nomes dos cliente de cada estado'''
consulta('''
SELECT estado, nome
FROM cliente;
''','')

,estado,nome
0,RJ,Ana Silva
1,RJ,Bruna Pereira
2,RJ,Túlio Nascimento
3,SP,Fernando Souza
4,SP,Lúcia Andrade
5,RJ,Rafael Costa
6,MS,Silvia Neves
7,RJ,Miguel Dias Santiago Norte Rodrigues
8,PE,Mariana Abrasante
9,BA,Anderson Sousa


In [8]:
consulta('''
SELECT estado, GROUP_CONCAT(nome SEPARATOR ',') AS nomes
FROM cliente
GROUP BY estado;
''','')

,estado,nomes
0,BA,Anderson Sousa
1,MS,Silvia Neves
2,PE,Mariana Abrasante
3,RJ,"Ana Silva,Bruna Pereira,Túlio Nascimento,Rafae..."
4,SP,"Fernando Souza,Lúcia Andrade"


In [10]:
'''
Listar os modelos de carros alugados por cada cliente:
'''

consulta('''
SELECT cliente.nome, GROUP_CONCAT(carro.modelo SEPARATOR ',') AS modelos
FROM cliente
JOIN aluguel ON cliente.codcliente = aluguel.codcliente
JOIN carro ON aluguel.codcarro = carro.codcarro
GROUP BY cliente.nome;
''','')

,nome,modelos
0,Ana Silva,"Ka,Polo,Polo"
1,Bruna Pereira,"Ka,Ka,Onix"
2,Lúcia Andrade,"Argo,Argo"
3,Rafael Costa,Kwid
4,Silvia Neves,Dolphin
5,Túlio Nascimento,"Ka,Argo"


In [11]:
'''
Mostre as marcas dos carros alugados por cada cliente
Exemplo:
Listar os modelos de carros alugados por cliente:
'''
consulta(
    '''
    SELECT cliente.nome, marca.marca
    FROM aluguel
    JOIN carro ON aluguel.codcarro = carro.codcarro
    JOIN marca ON marca.codmarca = carro.codmarca
    JOIN cliente ON aluguel.codcliente= cliente.codcliente
    ''',''
)

,nome,marca
0,Túlio Nascimento,Ford
1,Ana Silva,Ford
2,Bruna Pereira,Ford
3,Bruna Pereira,Ford
4,Lúcia Andrade,Fiat
5,Lúcia Andrade,Fiat
6,Túlio Nascimento,Fiat
7,Bruna Pereira,Chevrolet
8,Ana Silva,Volkswagen
9,Ana Silva,Volkswagen


In [12]:
'''
COM GROUP_CONCAT
'''

consulta('''
SELECT cliente.nome, GROUP_CONCAT(marca.marca SEPARATOR ',') AS marcas
FROM cliente
JOIN aluguel ON cliente.codcliente = aluguel.codcliente
JOIN carro ON aluguel.codcarro = carro.codcarro
JOIN marca ON carro.codmarca = marca.codmarca
GROUP BY cliente.nome;
    ''',''
)

,nome,marcas
0,Ana Silva,"Volkswagen,Volkswagen,Ford"
1,Bruna Pereira,"Ford,Ford,Chevrolet"
2,Lúcia Andrade,"Fiat,Fiat"
3,Rafael Costa,Renault
4,Silvia Neves,BYD
5,Túlio Nascimento,"Fiat,Ford"


In [13]:
'''
Exiba os nomes dos clientes e as datas em que alugaram carros, concatenando as
datas.
'''
consulta('''
SELECT cliente.nome,
GROUP_CONCAT(DATE_FORMAT(aluguel.data_aluguel, '%d-%m-%Y') SEPARATOR ',') AS datas
FROM aluguel
JOIN cliente ON aluguel.codcliente=cliente.codcliente
GROUP BY cliente.nome;
''','')

,nome,datas
0,Ana Silva,"05-04-2023,13-04-2023,15-04-2023"
1,Bruna Pereira,"02-04-2023,03-04-2023,04-04-2023"
2,Lúcia Andrade,"19-04-2023,21-04-2023"
3,Rafael Costa,19-03-2025
4,Silvia Neves,20-03-2025
5,Túlio Nascimento,"01-04-2023,25-04-2023"


In [14]:
consulta('''
SELECT * FROM aluguel
''','')

,codaluguel,codcliente,codcarro,data_aluguel
0,1,3,2,2023-04-01
1,2,2,1,2023-04-02
2,3,2,1,2023-04-03
3,4,2,3,2023-04-04
4,5,1,4,2023-04-05
5,6,1,4,2023-04-13
6,7,1,1,2023-04-15
7,8,5,2,2023-04-19
8,9,5,2,2023-04-21
9,10,3,1,2023-04-25


In [15]:
consulta('''
SHOW TABLES;
''','')

,Tables_in_Locadora
0,aluguel
1,carro
2,cliente
3,marca


In [16]:
consulta('''
SHOW TABLES;
''','')

,Tables_in_Locadora
0,aluguel
1,carro
2,cliente
3,marca


In [17]:
'''
Crie uma view que armazena a consulta de modelo e valor como tabela no
banco de dados
'''
consulta('''
CREATE VIEW modelo_valor AS
SELECT modelo, valor
FROM carro;
''','')

""


In [18]:
consulta('''
DESC modelo_valor;
''','')

,Field,Type,Null,Key,Default,Extra
0,modelo,varchar(30),NO,,None,
1,valor,float,YES,,None,


In [ ]:
'''
Uma VIEW é uma tabela virtual ou uma consulta armazenada. Essa tabela virtual, é criada através de um conjunto de
resultados de uma consulta, como por exemplo, sempre preciso executar uma query para realizar uma consulta no meu
banco de dados, em vez de sempre precisar realizar uma consulta complexa e que pode consumir mais recursos do meu
banco de dados, eu crio uma VIEW com este comando, onde uma tabela virtual é criada com o resultado deste comando.

Então a vantagem de utilizar a VIEW, é que ao invés de sempre rodar a query que é mais complexa, apenas será feito
um select mais simples na VIEW que já tem os dados armazenados na tabela virtual.

Uma view é uma forma de implementar segurança em banco de dados, ocultando informações a
determinados grupos de usuários.
 ■ Uma view não existe necessariamente na forma física – ela é considerada uma tabela virtual:
 ■ Há limitação em relação às operações de atualização que podem ser aplicadas;
 ■ Não há limitação em relação às operações de consulta.
 ■ As tabelas utilizadas na definição da visão são chamadas de tabelas definidoras de visão

'''

In [19]:
# Criando uma view juntando todos os elementos das tabelas carro e aluguel
consulta('''
CREATE OR REPLACE VIEW carro_aluguel AS
SELECT carro.*, aluguel.codaluguel, aluguel.codcliente, aluguel.data_aluguel
FROM carro
LEFT JOIN aluguel ON carro.codcarro = aluguel.codcarro;
''','')

""


In [20]:
consulta('''
SELECT * FROM carro_aluguel
''','')

,codcarro,codmarca,modelo,valor,codaluguel,codcliente,data_aluguel
0,1,1,Ka,100.0,2.0,2.0,2023-04-02
1,1,1,Ka,100.0,3.0,2.0,2023-04-03
2,1,1,Ka,100.0,7.0,1.0,2023-04-15
3,1,1,Ka,100.0,10.0,3.0,2023-04-25
4,2,2,Argo,150.0,1.0,3.0,2023-04-01
5,2,2,Argo,150.0,8.0,5.0,2023-04-19
6,2,2,Argo,150.0,9.0,5.0,2023-04-21
7,3,3,Onix,170.0,4.0,2.0,2023-04-04
8,4,4,Polo,150.0,5.0,1.0,2023-04-05
9,4,4,Polo,150.0,6.0,1.0,2023-04-13


In [21]:
'''
Utilizando a tabela cliente e a view carro_aluguel
faça uma consulta que exiba nome, modelo e data_aluguel
'''

consulta('''
SELECT cliente.nome, carro_aluguel.modelo, carro_aluguel.data_aluguel
FROM carro_aluguel
JOIN cliente ON carro_aluguel.codcliente = cliente.codcliente;
''','')

,nome,modelo,data_aluguel
0,Túlio Nascimento,Argo,2023-04-01
1,Bruna Pereira,Ka,2023-04-02
2,Bruna Pereira,Ka,2023-04-03
3,Bruna Pereira,Onix,2023-04-04
4,Ana Silva,Polo,2023-04-05
5,Ana Silva,Polo,2023-04-13
6,Ana Silva,Ka,2023-04-15
7,Lúcia Andrade,Argo,2023-04-19
8,Lúcia Andrade,Argo,2023-04-21
9,Túlio Nascimento,Ka,2023-04-25


In [22]:
'''
Utilizando a tabela cliente e a view carro_aluguel
faça uma consulta que exiba nome, modelo e data_aluguel
COM LEFT JOIN
'''
consulta('''
SELECT cliente.nome, carro_aluguel.modelo, carro_aluguel.data_aluguel
FROM carro_aluguel
LEFT JOIN cliente ON carro_aluguel.codcliente = cliente.codcliente;
''','')

,nome,modelo,data_aluguel
0,Bruna Pereira,Ka,2023-04-02
1,Bruna Pereira,Ka,2023-04-03
2,Ana Silva,Ka,2023-04-15
3,Túlio Nascimento,Ka,2023-04-25
4,Túlio Nascimento,Argo,2023-04-01
5,Lúcia Andrade,Argo,2023-04-19
6,Lúcia Andrade,Argo,2023-04-21
7,Bruna Pereira,Onix,2023-04-04
8,Ana Silva,Polo,2023-04-05
9,Ana Silva,Polo,2023-04-13


In [23]:
consulta('''
SELECT cliente.nome, carro_aluguel.modelo, carro_aluguel.data_aluguel
FROM carro_aluguel
RIGHT JOIN cliente ON carro_aluguel.codcliente = cliente.codcliente;
''','')

,nome,modelo,data_aluguel
0,Ana Silva,Polo,2023-04-05
1,Ana Silva,Polo,2023-04-13
2,Ana Silva,Ka,2023-04-15
3,Bruna Pereira,Ka,2023-04-02
4,Bruna Pereira,Ka,2023-04-03
5,Bruna Pereira,Onix,2023-04-04
6,Túlio Nascimento,Argo,2023-04-01
7,Túlio Nascimento,Ka,2023-04-25
8,Fernando Souza,None,None
9,Lúcia Andrade,Argo,2023-04-19
